Deep learning project code , Arabic dilects identification 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
'''
load data
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing
import io
import seaborn as sns
from sklearn import datasets
from sklearn import preprocessing #to normalize data
import re
import csv
print("hello world!")

from google.colab import drive
drive.mount('/gdrive')

import nltk # for processing texts
from nltk.corpus import stopwords # list of stop words
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

hello world!
Mounted at /gdrive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
train = pd.read_csv('/gdrive/My Drive/Project-DataSceince/train.csv', sep='\t',encoding = 'utf-16')
valid = pd.read_csv('/gdrive/My Drive/Project-DataSceince/valid.csv', sep='\t',encoding = 'utf-16')
test = pd.read_csv('/gdrive/My Drive/Project-DataSceince/test.csv', sep='\t',encoding = 'utf-16')

Text Normalization

In [4]:
#Explore the train data
train.head()


,text,label
0,بالإضافة لقيام معلمو الجيزة للذهاب إلي جريدة ا...,MSA
1,بعدين والله حرام تجي تلقى الي واقف عند الاشاره...,MSA
2,لمسه اليد مرتين واضحة جدا والحكم,DIAL_LEV
3,بخصوص الهاتريك عمرها ما راح تصير,DIAL_LEV
4,الله يجبر كسرهم ويرجع و لدهم اليوم قبل بكرى ،,DIAL_GLF


In [5]:
#Explore the valid data
valid.head()

,text,label
0,تصور كل ده كله . . تلاقيني باحبك أكتر .,DIAL_EGY
1,أكتر من روحي باحبك .,DIAL_EGY
2,وتصور حب المخلص لحبيبه اللي ولا بأيه يتغير .,DIAL_EGY
3,تصور كل ده كله . . تلاقيني باحبك أكتر .,DIAL_EGY
4,أكتر من روحي باحبك . . بحقيقي باحبك أكتر .,DIAL_EGY


In [14]:
import sys
import re
#sys.setdefaultencoding('utf-8')
##########################Clean Text Data #######################################
########################Global Variable Declaration##############################

#################################################################################

def normalize(sent):
    """clean data from any English char, emoticons, underscore, and repeated > 2
    str -> str"""
    p1 = re.compile('\W')
    p2 = re.compile('\s+')
    sent = ReplaceThreeOrMore(sent)
    sent = sent.replace('_', ' ')
    sent = re.sub(r'[A-Za-z0-9]', r'', sent)
    sent = re.sub(p1, ' ', sent)
    sent = re.sub(p2, ' ', sent)
    return sent
def ReplaceThreeOrMore(s):
    # pattern to look for three or more repetitions of any character, including
    # newlines.
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
def norm_alif(text):
    text = text.replace(u"\u0625", u"\u0627")  # HAMZA below, with LETTER ALEF
    #text = text.replace(u"\u0621", u"\u0627")  # HAMZA, with LETTER ALEF
    text = text.replace(u"\u0622", u"\u0627")  # ALEF WITH MADDA ABOVE, with LETTER ALEF
    #text = text.replace(u"\u0623", u"\u0627")  # ALEF WITH HAMZA ABOVE, with LETTER ALEF
    return text
def remove_unicode_diac(text):
    """Takes Arabic in utf-8 and returns same text without diac"""
    # Replace diacritics with nothing
    text = text.replace(u"\u064B", "")  # fatHatayn
    text = text.replace(u"\u064C", "")  # Dammatayn
    text = text.replace(u"\u064D", "")  # kasratayn
    text = text.replace(u"\u064E", "")  # fatHa
    text = text.replace(u"\u064F", "")  # Damma
    text = text.replace(u"\u0650", "")  # kasra
    text = text.replace(u"\u0651", "")  # shaddah
    text = text.replace(u"\u0652", "")  # sukuun
    text = text.replace(u"\u0670", "`")  # dagger 'alif
    return text
# def norm_taa(text):
#     text=text.replace(u"\u0629", u"\u0647") # taa' marbuuTa, with haa'
#     #text=text.replace(u"\u064A", u"\u0649") # yaa' with 'alif maqSuura
#     return text
# def norm_yaa(text):
#     if len(text)!=0:
#         if text[-1] == u"\u064A":
#             text = text.replace(u"\u064A", u"\u0649")  # yaa' with 'alif maqSuura
#     return text

def NormForWord2Vec(text):
    # text=norm_taa(text)
    # text=norm_yaa(text)
    text=norm_alif(text)
    return text

def remove_nonunicode2(Tweet):
    ## defining set of unicode ##
    #u""
    #Tweet=Tweet.decode("utf-8")
    UniLex={ ## This is list of all arabic unicode characters in addition to space (to separate words)
            u"\u0622",
            u"\u0626",
            u"\u0628",
            u"\u062a",
            u"\u062c",
            u"\u06af",
            u"\u062e",
            u"\u0630",
            u"\u0632",
            u"\u0634",
            u"\u0636",
            u"\u0638",
            u"\u063a",
            u"\u0640",
            u"\u0642",
            u"\u0644",
            u"\u0646",
            u"\u0648",
            u"\u064a",
            u"\u0670",
            u"\u067e",
            u"\u0686",
            u"\u0621",
            u"\u0623",
            u"\u0625",
            u"\u06a4",
            u"\u0627",
            u"\u0629",
            u"\u062b",
            u"\u062d",
            u"\u062f",
            u"\u0631",
            u"\u0633",
            u"\u0635",
            u"\u0637",
            u"\u0639",
            u"\u0641",
            u"\u0643",
            u"\u0645",
            u"\u0647",
            u"\u0649",
            u"\u0671",
            ' ',
            '\n'
          }
    fin_tweet=""
    for c in Tweet:
        if c in UniLex:
           fin_tweet=fin_tweet+c
    return fin_tweet

###### Heuristics Calculations ######
def diac_counter(text):
    #text=text.decode("utf-8")
    diac = [u"\u064B",u"\u064C", u"\u064D", u"\u064E", u"\u064F", u"\u0650", u"\u0651", u"\u0652", u"\u0670"]
    diac_count=0
    for d in diac:
        diac_count+=text.count(d)
#         if d in text:
#             print(d)
#             diac_count+=1
    return diac_count
def check_seed(list_seeds, text):
    ""
    for word in list_seeds:
        text = text.lower()
        if word.decode("utf-8") in text:
            return True
    return False
def EnglishCount(text):
    printable = ['e', 'a', 'o', 't', 'i']
    count = 0
    for ch in printable:
        count += text.count(ch.lower())
    return count
########################################



def eliminate_single_char_words(Tweet):
    parts = Tweet.split(" ")
    cleaned_line_parts = []
    for P in parts:
        if len(P) != 1:
            cleaned_line_parts.append(P)
    cleaned_line = ' '.join(cleaned_line_parts)
    return cleaned_line
def clean_unicode(Tweet):
    tweet=normalize(Tweet.strip("\n"))
    if len(tweet) !=0:
        sentence = []
        for word in tweet.split(" "):
            word = remove_unicode_diac(word)
            word = norm_alif(word)
            # word = norm_taa(word)
            # word = norm_yaa(word)
            word = normalize(word)
            word = Remove_stopword(word)
            sentence.append(word)
        tweet = ' '.join(sentence)
        tweet =remove_nonunicode2(tweet)
        tweet =eliminate_single_char_words(tweet)
    return tweet

def NormCorpusFinal(Tweet):
    tweet=KeepUniOnly(Tweet)
    tweet=NormForWord2Vec(tweet)
    return tweet

def KeepUniOnly(Tweet):## this one is without normalization
    tweet=Tweet.replace("# "," ")
    tweet=tweet.replace("#"," ")
    tweet=tweet.replace("_"," ")
    tweet=tweet.replace(u"\u0657"," ")
    tweet=tweet.replace("\n"," ")
    tweet=tweet.replace("["," ")
    tweet=tweet.replace("]"," ")
    tweet=tweet.replace("?"," ")
    tweet=tweet.replace("!"," ")
    tweet=remove_nonunicode2(tweet)
    tweet=eliminate_single_char_words(tweet)
    tweet=ReplaceThreeOrMore(tweet)
    return tweet

def get_charset(rawtext):
    chars = sorted(list(set(rawtext)))
    return chars

def DialectChecker(text):
    ##Based on Hueristics done by Hassan
    if (diac_counter(text)>5 or check_seed(list_seeds,text) or EnglishCount(text)>4 or "<URL>"  in text
        or text.count('#') >2 or '"'  in text or text.count('@') or "\"RT" in text or len(text.split(" ")) <7):
        return False
    else:
        return True

def Remove_stopword(text):

  text = ' '.join([word for word in text.split() if word not in stopwords.words("arabic")]) # remove stop word
  #text = re.sub(' +', ' ',text) # remove extra space
  #text = text.strip() #remove whitespaces
  return text

###############################################################

In [15]:
train.iloc[20000][0], train.iloc[20000][1]

('كل يوم نسمع صفقات انتقال اللاعبين من والى وبصفقات خيالية تصل الى مئات ملايين الدولارات وتنقل الصحافة الاخبار الى الرياضيين . . . لكن هل يوجد للاعبيناسوق انتقال منظمة ومحمية بالقانون الاردني وغير مخفية على الصحافةوخاصة مع تطبق نظام الاحتراف . . . ام ان الاتدية لازالت تمارس الضغط المعنوي والنفسي والمادي على اللاعبين حتى لايعرف الواحد راسة من رجلية . . . ويمارس حريتةالرياضية داخل النادي',
 'MSA')

In [16]:
norm_alif("اصيل ابو حق أ ئ ء~")
remove_nonunicode2("تاىبسشهيبللاشخسعيهىحشءةخئ بهلارسيعخؤشسىنحةمو الايتىنة  يهتؤخشنس GUHBDICNKMAX")

'تاىبسشهيبللاشخسعيهىحشءةخئ بهلارسيعخشسىنحةمو الايتىنة  يهتخشنس '

In [17]:
for row in range(0, 10):
  print (normalize(valid.iloc[row][0]) )

تصور كل ده كله تلاقيني باحبك أكتر 
أكتر من روحي باحبك 
وتصور حب المخلص لحبيبه اللي ولا بأيه يتغير 
أكتر من روحي باحبك بحقيقي باحبك أكتر 
وغنايا بكلمة أحبك بالنسبة لدرجة حبك رمز صغير مش أكتر 
وتصور حب الأزهار لربيعها يجي يزوق ويعطر 
حب الأطيار ف عششها لصغارها لحد ما تكبر 
حب الكروان للمغنى بالليل لوليف مستنظر 
حب الحران للنسمة ف الصيف والجو محرر 
أكتر من روحي باحبك اهداء الى كل محبى وعشاق نادى الزمالك احلى ناس فى الدنيا انشر ياناشر يا احلى


**Save**


In [18]:
print("train=", train.shape)
print("valid=", valid.shape)

train= (79594, 2)
valid= (9727, 2)


In [19]:
train.fillna("nan")
train.dropna(inplace=True)
print('Missing values in train data :',train.isnull().sum().sum())

Missing values in train data : 0


In [ ]:
for row in range(0, 500):
  print(clean_unicode(train.iloc[row][0]) )

In [21]:
valid.fillna("nan")
valid.dropna(inplace=True)
print('Missing values in valid data :',valid.isnull().sum().sum())

Missing values in valid data : 0


In [23]:
test.fillna("nan")
test.dropna(inplace=True)
print('Missing values in test data :',test.isnull().sum().sum())

Missing values in test data : 0


In [22]:
train.drop_duplicates(inplace = True)
valid.drop_duplicates(inplace = True)
test.drop_duplicates(inplace = True)



In [25]:
print("train=", train.shape)
# print("test=", test.shape)

train= (79590, 2)


In [34]:
#normalize train data , output = x,y

with open('/gdrive/My Drive/Project-DataSceince/Normalized_train.csv', 'w') as file:
    clean_country = csv.writer(file)
    clean_country.writerow(['tweet_content', 'label'])
    for row in  range(0, 79590):
     clean_country.writerow([clean_unicode(train.iloc[row][0]),train.iloc[row][1]])

In [27]:
print("valid=", valid.shape)


valid= (9727, 2)


In [30]:
#normalize valid data , output = x,y

with open('/gdrive/My Drive/Project-DataSceince/Normalized_valid.csv', 'w') as file:
    clean_country = csv.writer(file)
    clean_country.writerow(['tweet_content', 'label'])
    for row in  range(0, 9727):
     clean_country.writerow([clean_unicode(valid.iloc[row][0]),valid.iloc[row][1]])

In [31]:
print("test=", test.shape)


test= (10209, 2)


In [33]:
#normalize valid data , output = x,y

with open('/gdrive/My Drive/Project-DataSceince/Normalized_test.csv', 'w') as file:
    clean_country = csv.writer(file)
    clean_country.writerow(['tweet_content', 'label'])
    for row in  range(0, 10209):
     clean_country.writerow([clean_unicode(test.iloc[row][0]),test.iloc[row][1]])